
# Taller (3h) — FastAPI + Scikit-Learn orientado a investigación
**Objetivo:** Diseñar, entrenar y servir un modelo de ML **investigando** las decisiones técnicas clave.  
**Entrega:** API funcional (FastAPI) + breve informe (markdown dentro del notebook) justificando decisiones.

> Filosofía del taller: menos receta, más criterio. No hay una única respuesta correcta; lo evaluado es la **calidad del razonamiento**, la **limpieza de la implementación** y la **capacidad de probar** la API.



## Agenda sugerida (3h)
1) **Planteo del problema y dataset** (30–40 min)  
2) **Entrenamiento y persistencia** (40–50 min)  
3) **Diseño del contrato y API** (45–55 min)  
4) **Pruebas, errores y mejoras** (30–35 min)

### Reglas
- No borres los encabezados `TODO`. Agrega tu código debajo de cada bloque indicado.
- Documenta tus decisiones en la sección **Bitácora** al final.
- Puedes trabajar en equipo, pero cada entrega debe ser individual y original.



---
## 1) Selección de dataset y formulación del problema (investigación)
Elige **uno**:
- A) `sklearn.datasets.load_wine` (clasificación 3 clases, baseline rápido)
- B) Un dataset tabular de UCI, Kaggle u otra fuente **citable** (debe ser pequeño/mediano y con licencia apta)
- C) Un CSV propio (explica origen y variables)

**Requisitos mínimos:**
- Problema de **clasificación** o **regresión** tabular
- Al menos **6 features numéricas** (puedes convertir categóricas)
- Justifica por qué es un buen caso para servir vía API

**Entrega (en esta celda, texto breve):** Describe el dataset, objetivo, variables y métrica principal.



### 1.1 Carga y exploración (TODO)
- Carga el dataset (pd.read_csv o loader de sklearn).
- Muestra `head()`, `describe()` y verifica nulos/outliers.
- Selecciona `X` (features) y `y` (target); explica tu elección.


In [1]:
# TODO: Carga y EDA mínima
import pandas as pd

# from sklearn.datasets import load_wine

# Ejemplo de estructura (reemplaza con tu fuente):
# df = pd.read_csv("TU_DATASET.csv")
# ---- TU CÓDIGO AQUÍ ----
raise NotImplementedError("Implementa la carga y exploración del dataset")

NotImplementedError: Implementa la carga y exploración del dataset


---
## 2) Entrenamiento y persistencia del modelo (investigación)
Toma decisiones y **justifícalas**:
- ¿Modelo base? (p. ej. `LogisticRegression`, `RandomForest`, `XGBoost` si lo instalas)
- ¿Preprocesamiento? (escala, imputación, OneHot, etc.)
- ¿Validación? (`train_test_split` vs `cross_val_score`)
- ¿Métrica? (clasificación: accuracy/F1; regresión: RMSE/MAE…)

**Requisito:** empaqueta tu flujo en un `Pipeline` de sklearn y **persiste** el modelo y columnas (joblib + JSON).


In [ ]:
# TODO: split, pipeline, training, evaluación y persistencia
from pathlib import Path
import json, joblib

ARTIFACTS_DIR = Path("artifacts")
ARTIFACTS_DIR.mkdir(exist_ok=True)

# Variables esperadas:
# X, y = ...
# pipeline = ...
# métricas calculadas en 'metrics' (dict)

# ---- TU CÓDIGO AQUÍ ----
raise NotImplementedError("Implementa split, pipeline, entrenamiento y evaluación")

# TODO: Persistir
# joblib.dump(pipeline, ARTIFACTS_DIR / "model.joblib")
# json.dump({"feature_columns": list(X.columns)}, open(ARTIFACTS_DIR / "feature_columns.json", "w"))
# json.dump({"metrics": metrics}, open(ARTIFACTS_DIR / "metadata.json", "w"))


---
## 3) Diseño del contrato de la API (investigación)
Define **endpoints mínimos**:
- `GET /health` (estado, versión del modelo, métrica)
- `POST /predict` (un registro)
- `POST /predict-batch` (lista de registros)

**Decisiones a justificar:**
- ¿Qué validaciones aplicas en `Pydantic`? (rango, tipos, campos extra)
- ¿Cómo garantizas el **orden** de columnas?
- ¿Qué devuelves además de la predicción? (probabilidades, latencia, advertencias)


In [ ]:
# TODO: Generar un esquema dinámico a partir de las columnas persistidas (opcional pero recomendado)
# Carga feature_columns.json y crea un dict para generar ejemplos de payload
import json, os
from pathlib import Path

ARTIFACTS_DIR = Path("artifacts")
feat_path = ARTIFACTS_DIR / "feature_columns.json"
# ---- TU CÓDIGO AQUÍ ----
raise NotImplementedError(
    "Construye un ejemplo de payload a partir de las columnas persistidas"
)


---
## 4) Implementación de FastAPI (investigación)
Crea un archivo `app.py` con:
- Carga perezosa de `model.joblib` y `feature_columns.json`
- Esquemas Pydantic (v2)
- Endpoints `/health`, `/predict`, `/predict-batch`
- Manejo de errores con `HTTPException` y mensajes claros

**Pistas** (no copiar/pegar sin entender):
- `model = joblib.load(...)`
- `class Sample(BaseModel): ...`
- `model.predict` y/o `model.predict_proba`
- Retornar JSON con `dict | BaseModel`

**Requisito:** esta celda **debe** escribir `app.py` con al menos la estructura básica.


In [2]:
# TODO: Escribir app.py
from textwrap import dedent
from pathlib import Path

app_code = dedent(
    """
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, ConfigDict, Field
from typing import List, Dict
import joblib, json, time
from pathlib import Path

APP_VERSION = "0.1.0"
ARTIFACTS_DIR = Path("artifacts")
MODEL_PATH = ARTIFACTS_DIR / "model.joblib"
COLUMNS_PATH = ARTIFACTS_DIR / "feature_columns.json"
META_PATH = ARTIFACTS_DIR / "metadata.json"

app = FastAPI(title="ML API", version=APP_VERSION)

# TODO: Define tus modelos Pydantic aquí
class Sample(BaseModel):
    model_config = ConfigDict(extra="forbid")
    # ---- COMPLETA CON TUS CAMPOS SEGÚN feature_columns.json ----
    # ejemplo: feature_x: float = Field(..., ge=0)

class PredictionOut(BaseModel):
    # ---- COMPLETA: qué devuelves? label / score / probs / latencia ----
    pass

@app.on_event("startup")
def load_artifacts():
    global model, columns, meta
    if not MODEL_PATH.exists():
        raise RuntimeError("Modelo no encontrado. Entrena y exporta primero.")
    model = joblib.load(MODEL_PATH)
    columns = json.loads(COLUMNS_PATH.read_text())["feature_columns"]
    meta = json.loads(META_PATH.read_text()) if META_PATH.exists() else {}

@app.get("/health")
def health():
    return {
        "status": "ok",
        "version": APP_VERSION,
        "metrics": meta.get("metrics"),
        "n_features": len(columns)
    }

@app.post("/predict")
def predict(sample: Sample):
    try:
        start = time.perf_counter()
        # TODO: transforma 'sample' a lista en el orden de 'columns'
        # y haz model.predict / predict_proba
        # ---- TU CÓDIGO AQUÍ ----
        raise NotImplementedError
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

@app.post("/predict-batch")
def predict_batch(samples: List[Sample]):
    try:
        # TODO: similar a /predict pero para lista
        # ---- TU CÓDIGO AQUÍ ----
        raise NotImplementedError
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))
"""
)

Path("app.py").write_text(app_code)
"app.py escrito (plantilla con TODOs)"

'app.py escrito (plantilla con TODOs)'


### 4.1 Ejecutar el servidor
```bash
uvicorn app:app --reload --port 8000
```
Abre `http://127.0.0.1:8000/docs` y prueba manualmente. Registra resultados.



---
## 5) Pruebas y casos límite (investigación)
Define y ejecuta **al menos 6** pruebas:
- 3 válidas (predict y batch)
- 3 inválidas (campo faltante, tipo incorrecto, campo extra, etc.)

Incluye código de prueba y captura de respuestas.


In [ ]:
# TODO: pruebas con requests (server debe estar corriendo)
# import requests, json
# payload_valido = {...}
# r = requests.post("http://127.0.0.1:8000/predict", json=payload_valido, timeout=5)
# print(r.status_code, r.json())
raise NotImplementedError("Escribe tus pruebas de cliente aquí")


---
## 6) (Opcional) Observabilidad y despliegue (investigación)
- Middleware de latencia y logger estructurado (añade header `X-Process-Time-ms`)
- Manejo de warnings cuando la entrada está fuera de rango esperado
- Dockerfile mínimo para empaquetar y correr localmente


In [ ]:
# TODO (opcional): pega aquí snippets de middleware o Dockerfile que diseñes
# class TimingMiddleware(...): ...
# FROM python:3.11-slim
pass


---
## 7) Criterios de evaluación (rúbrica breve)
- **Justificación técnica (25%)**: dataset, métrica, modelo y preprocesamiento argumentados.
- **Calidad del pipeline (20%)**: reproducibilidad y limpieza (Pipeline, persistencia correcta).
- **Contrato y validaciones (25%)**: Pydantic coherente, errores claros, orden de features garantizado.
- **Pruebas (20%)**: variedad de casos, evidencia de resultados y manejo de fallos.
- **Código y documentación (10%)**: legibilidad, estructura y claridad de mensajes.

---
## Bitácora de decisiones (responde aquí)
- Dataset y objetivo:
- Selección de features/target:
- Modelo y preprocesamiento:
- Métrica principal y resultados:
- Decisiones de contrato (payload, validaciones, respuestas):
- Observabilidad y pruebas:
- Lecciones aprendidas:
